In [1]:
# -*- coding: utf-8 -*-

# don
import pandas as pd
import numpy as np
import librosa
import os
import zipfile
import csv
import shutil
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.metrics import (
    roc_auc_score,        
)


In [2]:
"""
Script for extracting data from https://figshare.com/articles/dataset/Sounds_of_the_Eleutherodactylus_frog_community_from_Puerto_Rico/806302?file=3104183
Unzips all the zips from root
Simply Unzip downloaded file and provide path root of folder
"""


def extract_zip_file(file_path, extract_to):
    """
    Extracts a single zip file to a specified directory.

    Args:
        file_path (str): Path to the zip file.
        extract_to (str): Path to the directory where the zip file will be extracted.
    """
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
        # print(f"{os.path.basename(file_path)} extracted to {os.path.abspath(extract_to)}")


def extract_zip_files(zip_folder, extract_to):
    """
    Extracts all zip files from a folder to a specified directory using threading.

    Args:
        zip_folder (str): Path to the folder containing zip files.
        extract_to (str): Path to the directory where zip files will be extracted.
    """
    # Make sure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    data_file = "FrequencyRange_by_species_and_site_Averages.csv"
    shutil.copyfile(os.path.join(zip_folder, data_file) , os.path.join(extract_to, data_file))

    # List all zip files in the folder
    zip_files = [
        os.path.join(zip_folder, item)
        for item in os.listdir(zip_folder)
        if item.endswith(".zip")
    ]

    # Use ThreadPoolExecutor to extract zip files concurrently
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(extract_zip_file, zip_file, extract_to)
            for zip_file in zip_files
        ]

        # Wait for all futures to complete
        for future in futures:
            future.result()
    
    print("Done.")

In [4]:
filepath = "/home/edwinc/Downloads/806302"
ExtractTo = "/home/edwinc/Downloads/806302/Extracted"
extract_zip_files(filepath, ExtractTo)

Done.


In [5]:
def readAveragesData(path: str):

    data = []
    # Read the CSV file and store the data in a list of dictionaries
    with open(path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)

    return data


def prepare_csv(data_dir, output) -> None:

    
    averagesData = readAveragesData(
        os.path.join(data_dir, "FrequencyRange_by_species_and_site_Averages.csv")
    )

    data = []

    # Iterate through each subfolder
    for siteDataSet in os.listdir(data_dir):
        site_folder = os.path.join(data_dir, siteDataSet)
        if os.path.isdir(site_folder):
            # example siteId  "Site01-1" such that the 4-6 index represents the site id; in this case 01
            siteId = int(siteDataSet[4:6])
            SiteData = [
                averageClassification
                for averageClassification in averagesData
                if int(averageClassification["SiteID"]) == siteId
            ]

            classifications = ", ".join(
                [classification["Species"] for classification in SiteData]
            )
            for audio_recording in os.listdir(site_folder):
                if audio_recording.endswith(".wav"):
                    audio_recording_abs_path = os.path.abspath(
                        os.path.join(site_folder, audio_recording)
                    )

                    data.append([siteId, audio_recording_abs_path, classifications])

    # Create DataFrame
    df = pd.DataFrame(
        data,
        columns=[
            "siteId",
            "filename",
            "species",
        ],
    )

    df.to_csv(output, index=False)

In [10]:
data_dir = ExtractTo
output = "machine_learning/processed/processed.csv"
prepare_csv(data_dir, output)

In [11]:
def extract_features(file_path):
    """
    Extract features from audio file using librosa.

    Args:
        file_path (str): Path to the audio file.

    Returns:
        np.array: Extracted features.
    """
    audio, sr = librosa.load(file_path)
    result = np.array([])

    # MFCC
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr).T, axis=0)
    result = np.hstack((result, mfccs))

    # Chroma
    stft = np.abs(librosa.stft(audio))
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    result = np.hstack((result, chroma))

    # Mel-scaled spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr).T, axis=0)
    result = np.hstack((result, mel))

    return result


def process_data(data_csv_path):
    """Read Csv with fileanme and generate spectrogram for each sample

    Returns:
        DataFrame: dataframe with all data
    """
    # data_csv_path = sys.argv[1]

    df = pd.read_csv(data_csv_path)

    # Initialize a list to store the results
    spectrograms = []

    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(extract_features, row["filename"])
            for _, row in df.iterrows()
        ]

        for future in as_completed(futures):
            try:
                spectrogram = future.result()
                spectrograms.append(spectrogram)
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    # Convert the list of spectrograms into a DataFrame
    spectrogram_df = pd.DataFrame(spectrograms)

    # Concatenate the original DataFrame with the new DataFrame containing spectrograms
    df = pd.concat([df, spectrogram_df], axis=1)

    return df

In [12]:
csv_dir = output
df = process_data(csv_dir)
df.head()

,siteId,filename,species,0,1,2,3,4,5,6,...,150,151,152,153,154,155,156,157,158,159
0,3,/home/edwinc/Downloads/806302/Extracted/Site03...,"E. coqui - co, E. coqui - qui, E. gryllus, E. ...",-204.848251,131.633331,-55.604843,64.099342,-0.086645,4.280726,28.252939,...,0.000021,0.000017,0.000015,0.000013,0.000011,0.000009,0.000008,0.000006,2.840367e-06,2.688746e-07
1,3,/home/edwinc/Downloads/806302/Extracted/Site03...,"E. coqui - co, E. coqui - qui, E. gryllus, E. ...",-209.817902,145.711716,-62.136204,58.966694,-2.604534,3.186882,28.051863,...,0.000009,0.000008,0.000007,0.000006,0.000005,0.000004,0.000003,0.000002,1.123617e-06,9.405731e-08
2,3,/home/edwinc/Downloads/806302/Extracted/Site03...,"E. coqui - co, E. coqui - qui, E. gryllus, E. ...",-219.265167,116.345963,-58.627560,62.047398,-6.808269,10.797720,26.177841,...,0.000037,0.000032,0.000027,0.000023,0.000020,0.000017,0.000015,0.000011,4.967286e-06,4.306051e-07
3,3,/home/edwinc/Downloads/806302/Extracted/Site03...,"E. coqui - co, E. coqui - qui, E. gryllus, E. ...",-204.993912,134.253296,-38.925591,61.476059,-17.941975,-3.416336,31.911119,...,0.000030,0.000024,0.000020,0.000017,0.000015,0.000012,0.000010,0.000008,4.315103e-06,4.778029e-07
4,3,/home/edwinc/Downloads/806302/Extracted/Site03...,"E. coqui - co, E. coqui - qui, E. gryllus, E. ...",-229.271957,135.000778,-58.928051,47.495621,2.579341,5.440523,29.769083,...,0.000006,0.000005,0.000005,0.000005,0.000004,0.000003,0.000002,0.000002,7.509618e-07,8.347875e-08


In [ ]:
x = df.drop(
    columns=["filename", "species", "siteId"]
)  # Adjust this to include only feature columns
# Convert all column names to strings
x.columns = x.columns.astype(str)

x

KeyError: "['sideId'] not found in axis"

In [14]:
y = df["species"]

# Encode the target labels as integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

le_bron = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
le_bron


{'E. coqui - co, E. coqui - qui': np.int64(0),
 'E. coqui - co, E. coqui - qui, E. gryllus, E. locustus': np.int64(1),
 'E. coqui - co, E. coqui - qui, E. gryllus, E. portoricensis - co, E. portoricensis - qui, E. unicolor': np.int64(2),
 'E. coqui - co, E. coqui - qui, E. hedricki': np.int64(3),
 'E. coqui - co, E. coqui - qui, E. hedricki, E. portoricensis - co, E. portoricensis - qui': np.int64(4),
 'E. coqui - co, E. coqui - qui, E. hedricki, E. portoricensis - co, E. portoricensis - qui, E. unicolor': np.int64(5),
 'E. coqui - co, E. coqui - qui, E. portoricensis - co, E. portoricensis - qui, E. richmondi': np.int64(6),
 'E. coqui - co, E. coqui - qui, E. portoricensis - co, E. portoricensis - qui, E. unicolor': np.int64(7),
 'E. coqui - co, E. coqui - qui, E. richmondi': np.int64(8),
 'E. coqui - co, E. coqui - qui, E. richmondi, E. wightmanae': np.int64(9),
 'E. coqui - co, E. coqui - qui, E. wightmanae': np.int64(10)}

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

classifier = RandomForestClassifier(n_estimators=600, max_depth=18, min_samples_leaf=3)

classifier.fit(x_train, y_train)

y_pred = classifier.predict_proba(
    x_test,
)

accuracy = roc_auc_score(y_test, y_pred, multi_class="ovr")
print("Accuracy :", accuracy)




Accuracy : 0.9948199735973198


In [ ]:
import pickle
with open("../backend/trainedRF.pkl", "wb") as f:
    pickle.dump(classifier, f)